In [3]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp__stocks.csv')
stocks

""


In [4]:
from secrets import IEX_CLOUD_API_TOKEN

ImportError: cannot import name 'IEX_CLOUD_API_TOKEN' from 'secrets' (C:\Users\rtg54\anaconda3\envs\dev\lib\secrets.py)

In [12]:
#Making Our First API Call
#Market Capitalization for each stock
#Price of each stock

symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com//stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)


NameError: name 'IEX_CLOUD_API_TOKEN' is not defined

In [15]:
#python dictonary called dictonary
#example for my memory
dictionary = {
       'a' : 1,
        'b' : 2
}
dictionary['a']

1

In [17]:
#parce price
#parce market cap
price = data['latestPrice']
print(price)
market_cap = ['marketCap']
print(market_cap/1000000000)


NameError: name 'data' is not defined

In [20]:
#Scale all the process to all the stocks
#Data to a Pandas DataFrame
#0000 row added for visual effect
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of shares to Buy']
final_dataframe = pd.DataFrame([[0,0,0,0,]],columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of shares to Buy
0,0,0,0,0


In [24]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    #which columns to add this data too
    index = my_columns
    ),
    #Necessary when appending data to pandas data frame
    ignore_index=True
)

NameError: name 'price' is not defined

In [25]:
#loop through the tickers in our list of stocks
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com//stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    #this line is very slow
    data = requests.get(api_url).json()
    #final_dataframe.apend( Was the initial try
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index = my_columns),
    ignore_index = True
    )

NameError: name 'stocks' is not defined

In [26]:
#using Batch API calls to improve performance
def chunks(lst, n):
    """Yield successive n-sized chunks from 1st."""
    for i in range(0, len(lst), n):
        yield list[i:i + n]

In [32]:
symbol_groups = list(chuncks(stocks['Ticker'], 100)
#symobl_groups
for i in range(0, len(symbol_groups)):
    symbol_strings.apend(','.join(symbol_groups[i]))
    #print(symbol_groups[i])  
final_dataframe = pd.DataFrame(colums = my_colums)
for symbol_string in symbol_strings:
    #after first ? the rest have to be ampersands &
    batch_api_call_url = f'https://sandbox.iexapis.com//stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index = my_columns),
            ignore_index = True
        )
final_dataframe   

SyntaxError: invalid syntax (256415886.py, line 3)

In [38]:
#Calculating the number of Shares to Buy #start by creating an input
#try and except is to prevent strings from breaking it
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
    print(val)
except:
    print("That's not a number! \nPlease try again:")
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

Enter the value of your portfolio: 100000


100000.0


In [41]:
postion_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])

#print(position_size)
#number_of_apple_shares = position_size/500
#Math to round down
#print(math.floor(number_of_apple_shares))

ZeroDivisionError: float division by zero

In [42]:
#creating the formats we'll Need for our .xlsx file
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = Flase)

NameError: name 'Flase' is not defined

In [45]:
#creating the formats we'll need for our .xlsx file
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'bg_color' : background_color,
        'border' : 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format' : '$0.00',
        'bg_color' : background_color,
        'border' : 1
    }
)

In [47]:
#Applying the formats to the columns of our .xlsx file
#writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
#writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
#writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
#writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)

#writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
#writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
#writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
#writer.sheets['Recommended Trades'].write('D1', 'Ticker', integer_format)

KeyError: 'Recommended Trades'

In [49]:
column_formats = {
    'A':['Ticker', string_format],
    'B':['Stock Price' , dollar_format],
    'C':['Market Capitalization', dollar_format],
    'D':['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    #Two Loops for previous to clean cold
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
writer.save()

KeyError: 'Recommended Trades'